In [3]:
from process_bigraph import Process

import cobra
from cobra.io import load_model

model = load_model("textbook")

In [2]:
class DynamicFBA(Process):
    """
    Performs dynamic FBA.

    Parameters:
    - model: The metabolic model for the simulation.
    - kinetic_params: Kinetic parameters (Km and Vmax) for each substrate.
    - biomass_reaction: The identifier for the biomass reaction in the model.
    - substrate_update_reactions: A dictionary mapping substrates to their update reactions.
    - biomass_identifier: The identifier for biomass in the current state.

    Returns:
    - Updated state after the timestep.
    """
    
    config_schema = {
        'model_file': 'Any',
        'kinetic_params': 'Any',
        'biomass_reaction': 'Any',
        'biomass_identifier': 'Any',
    }
  
    def __init__(self, config):
        super().__init__(config)
        self.model = cobra.io.read_sbml_model(self.config['model_file'])
        
    def inputs(self):
        return {}
    
    def outputs(self):
        return {}
    
    # TODO -- can we just put the inputs/outputs directly in the function?
    def update(self, state, interval):
        updated_state = state.copy()
        
        for substrate, reaction_id in substrate_update_reactions.items():
            Km, Vmax = kinetic_params[substrate]
            substrate_concentration = updated_state[substrate]
            uptake_rate = Vmax * substrate_concentration / (Km + substrate_concentration)
            self.model.reactions.get_by_id(reaction_id).lower_bound = -uptake_rate
    
        solution = self.model.optimize()
        if solution.status == 'optimal':
            current_biomass = state[self.config['biomass_identifier']]
            biomass_growth_rate = solution.fluxes[self.config['biomass_reaction']]
            updated_state[self.config['biomass_identifier']] += biomass_growth_rate * current_biomass * interval
    
            for substrate, reaction_id in substrate_update_reactions.items():
                flux = solution.fluxes[reaction_id]
                updated_state[substrate] = max(updated_state[substrate] + flux * current_biomass * interval, 0)
        else:
            # Handle non-optimal solutions if necessary
            pass
    
        return updated_state

In [2]:
model.reactions.EX_o2_e.lower_bound = -2  # Limiting oxygen uptake
model.reactions.ATPM.lower_bound = 1     # Setting lower bound for ATP maintenance
model.reactions.ATPM.upper_bound = 1     # Setting upper bound for ATP maintenance

# Define initial conditions, kinetic parameters, and other necessary inputs
initial_conditions = {
    'biomass': 0.1,  # Initial biomass concentration
    'glucose': 20.0, # Initial glucose concentration
    'acetate': 0.0   # Initial acetate concentration
}
kinetic_params = {
    'glucose': (0.5, 2), # Km and Vmax for glucose
    'acetate': (0.5, 2)  # Km and Vmax for acetate
}
substrate_update_reactions = {
    'glucose': 'EX_glc__D_e',  # Exchange reaction ID for glucose
    'acetate': 'EX_ac_e'       # Exchange reaction ID for acetate
}

# simulation conditions
t_n = 100 # number of time points
dt = 0.5  # Time step, matching your setup
time_points = list(range(0, t_n))  # Simulation time points, matching your setup

# Make sure to adjust the perform_dfba function if needed to initialize result arrays with sufficient size
dfba_results = perform_dfba(
    model, 
    initial_conditions, 
    kinetic_params, 
    time_points, 
    'Biomass_Ecoli_core', 
    substrate_update_reactions, 
    dt,
    'biomass'
)